In [1]:
import json, os, pprint, time, re
from urllib import parse
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
ua = UserAgent(cache=True)

In [8]:
listData = []
url = 'https://www.gutenberg.org/browse/languages/zh'
headers = {
    'user-agent': ua.random
}
folderPath = 'gutenberg3'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [9]:
# 取得中文書的主要連結
def getMainLinks():
    res=req.get(url, headers = headers)
    soup=bs(res.text,"lxml")
    mainLink=soup.select("li.pgdbetext a")
    for a in mainLink:
        regex=(r"\b[\u2E80-\u9FFF]+")  
        ch_book=re.match(regex,a.get_text()) # 使用正規表達式,篩選書名是中文的
        if ch_book!=None:
            listData.append({
                "Book":a.get_text(),
                "Link":'https://www.gutenberg.org/'+a['href'],
                "Sub":[]
            })
    print("getMainLinks()成功")

In [10]:
# 取得所有中文書的獨立連結
def getSubLinks():
    for index, i in enumerate(range(len(listData))):
        if index>300:
            break
        res=req.get(listData[i]['Link'], headers = headers)
        soup_=bs(res.text,"lxml")
        subLink=soup_.select("td[content='text/plain; charset=utf-8']>a")
        for a in subLink:
            if len(subLink)>0:
                listData[i]["Sub"].append({
                "SubLink":'https://www.gutenberg.org/'+a['href']
                })
            else: continue
    print("getSubLinks()成功")

In [13]:
#將中文書內容寫入text檔
def writeTxt():
    for index, i in enumerate(range(len(listData))):
        if index>300:
            break
        for j in range(len(listData[i]["Sub"])):
            res= req.get(listData[i]["Sub"][j]["SubLink"], headers=headers)
            res.encoding='utf-8'
            soup__=bs(res.text,"lxml")
            body=soup__.select_one("body")
            content=body.get_text()
            content=re.findall(r"[\u2E80-\u9FFF]",content) # 只取內容中文的部分
            content=''.join(content)                       # 將re結果的listˋ轉成string
            Book=listData[i]['Book']
            Book=re.sub(r"\s|\:", '_',Book)                #將書名中有空格或特殊自元
            filename=f"{Book}.txt"
            if content != None:
                with open(f"{folderPath}/{filename}","w", encoding="utf-8") as file:
                    file.write(content)
    print("writeTxt()成功")

In [14]:
# 主程式
if __name__=="__main__":
    time1=time.time()
    getMainLinks()
    getSubLinks()
    writeTxt()
    print("程式成功")
    print(f"總運行時間:{time.time()-time1}")

getMainLinks()成功
getSubLinks()成功
writeTxt()成功
程式成功
總運行時間:1216.2757749557495
